In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install openpyxl

In [ ]:
pip install geopandas

In [ ]:
pip install plotly geopandas

In [ ]:
import geopandas as gpd
import os

# Path to the directory containing .shp files
shapefile_dir = '/content/drive/MyDrive/MIT805_Exam/Shape_Data/zaf_admbnda_adm3_sadb_ocha_20201109.shp'

# Dictionary to store each shapefile by name
shapefiles = {}
shapefiles[file] = gpd.read_file(shapefile_dir)

In [ ]:
import pandas as pd

# Merge shapefiles with the same structure
merged_shapefile = pd.concat(shapefiles.values(), ignore_index=True)
merged_gdf = gpd.GeoDataFrame(merged_shapefile, geometry='geometry')


In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Step 1: Load Health Facility Data and Convert to GeoDataFrame
# Assuming health facilities data is in CSV format with columns 'Lat' and 'Long'
facilities_df = pd.read_excel('/content/drive/MyDrive/MIT805_Exam/health_facilities.xlsx')  # Adjust path as needed
facilities_df = facilities_df[facilities_df['Country'].str.strip().str.lower() == 'south africa']

# Create geometry column from Lat and Long
geometry = [Point(xy) for xy in zip(facilities_df['Long'], facilities_df['Lat'])]
facilities_gdf = gpd.GeoDataFrame(facilities_df, geometry=geometry)

# Step 2: Plot, adjusting for 'Population' if it exists
fig, ax = plt.subplots(figsize=(12, 10))
if 'Population' in merged_gdf.columns:
    merged_gdf.plot(column='Population', cmap="OrRd", legend=True,
                    legend_kwds={"label": "Population", "orientation": "horizontal"}, ax=ax)
else:
    print("Warning: 'Population' column not found. Plotting without population data.")
    merged_gdf.plot(ax=ax, legend=True)

# Overlay health facilities as blue points
facilities_gdf.plot(ax=ax, color='blue', alpha=0.7, markersize=5, label='Health Facilities')

ax.set_title("Population Distribution with Health Facilities")
plt.axis('off')  # Hide axis for a cleaner map
plt.legend()
plt.show()



In [ ]:
import geopandas as gpd
import plotly.express as px
import pandas as pd

# Load shapefile and facilities data
merged_gdf = gpd.read_file("/content/drive/MyDrive/MIT805_Exam/Shape_Data/zaf_admbnda_adm3_sadb_ocha_20201109.shp")  # Replace with your path
facilities_df = pd.read_excel("/content/drive/MyDrive/MIT805_Exam/health_facilities.xlsx")  # Replace with your path
facilities_df = facilities_df[facilities_df['Country'].str.strip().str.lower() == 'south africa']

# Set CRS if missing
if merged_gdf.crs is None:
    merged_gdf.set_crs("EPSG:4326", inplace=True)  # Set CRS to EPSG:4326 if unknown

# Convert facilities data to GeoDataFrame with Points
facilities_gdf = gpd.GeoDataFrame(
    facilities_df,
    geometry=gpd.points_from_xy(facilities_df['Long'], facilities_df['Lat'])
)

# Set CRS for facilities_gdf if missing
if facilities_gdf.crs is None:
    facilities_gdf.set_crs("EPSG:4326", inplace=True)  # Set CRS to EPSG:4326 if unknown

# Ensure both GeoDataFrames use the same CRS
merged_gdf = merged_gdf.to_crs("EPSG:4326")
facilities_gdf = facilities_gdf.to_crs("EPSG:4326")

# Convert merged_gdf to GeoJSON format for use in Plotly
geojson_data = merged_gdf.__geo_interface__

# Interactive map using Plotly Express
fig = px.choropleth_mapbox(
    merged_gdf,
    geojson=geojson_data,
    locations=merged_gdf.index,
    color="Population" if "Population" in merged_gdf.columns else None,  # Color by population if available
    hover_name="ADM1_EN",  # Replace with a column name for hover info
    hover_data={"Population": True} if "Population" in merged_gdf.columns else {},
    mapbox_style="carto-positron",
    center={"lat": -30.5595, "lon": 22.9375},  # Center on South Africa
    zoom=5,
    opacity=0.5,
    labels={"Population": "Population"}
)

# Add health facilities as scatter points
fig.add_scattermapbox(
    lat=facilities_gdf.geometry.y,
    lon=facilities_gdf.geometry.x,
    mode='markers',
    marker=dict(size=6, color='blue'),
    name='Health Facilities',
    hoverinfo='text',
    text=facilities_df['Facility_n']  # Adjust to the column you want for hover text
)

# Update layout for better visualization
fig.update_layout(
    title="Interactive Population Distribution with Health Facilities",
    margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from PIL import Image

# Load the GeoTIFF file
with rasterio.open("/content/drive/MyDrive/MIT805_Exam/population_density.tif") as src:
    # Read the first band and mask no-data values
    tiff_data = src.read(1)
    tiff_data = np.where(tiff_data == -99999.0, np.nan, tiff_data)

# Option 1: Normalize data based on percentiles for enhanced visibility
vmin, vmax = 0, np.nanpercentile(tiff_data, 95)  # Use 95th percentile for upper bound
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
tiff_data_normalized = norm(tiff_data)

# Option 2: Apply a color map (e.g., "inferno" or any other you prefer)
cmap = plt.cm.inferno  # Choose your color map (e.g., "viridis", "plasma", "magma")
rgba_img = cmap(tiff_data_normalized)  # This creates an RGBA (Red, Green, Blue, Alpha) image

# Convert RGBA to RGB by removing the alpha channel and save it as an image
rgb_img = (rgba_img[:, :, :3] * 255).astype(np.uint8)  # Scale to 0-255 for RGB
image = Image.fromarray(rgb_img)

# Save the colorized image if needed
image.save("population_density_colored.png")

# Display the image
plt.imshow(rgb_img)
plt.title("Colorized Population Density (using Inferno)")
plt.axis("off")
plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import matplotlib.colors as mcolors
from rasterio.mask import mask
from shapely.geometry import Point, box
import pandas as pd

# Load shapefile and facilities data
merged_gdf = gpd.read_file("/content/drive/MyDrive/MIT805_Exam/Shape_Data/zaf_admbnda_adm3_sadb_ocha_20201109.shp")
facilities_df = pd.read_excel("/content/drive/MyDrive/MIT805_Exam/health_facilities.xlsx")
facilities_df = facilities_df[facilities_df['Country'].str.strip().str.lower() == 'south africa']

# Convert facilities_df to a GeoDataFrame with geometry
facilities_df['geometry'] = facilities_df.apply(lambda row: Point(row['Long'], row['Lat']), axis=1)
facilities_gdf = gpd.GeoDataFrame(facilities_df, geometry='geometry')

# Filter facilities_gdf to keep only points within South Africa's bounding box
sa_bounds = {
    "lat_min": -35,
    "lat_max": -22,
    "lon_min": 16,
    "lon_max": 33
}
facilities_gdf = facilities_gdf[
    (facilities_gdf.geometry.x >= sa_bounds["lon_min"]) &
    (facilities_gdf.geometry.x <= sa_bounds["lon_max"]) &
    (facilities_gdf.geometry.y >= sa_bounds["lat_min"]) &
    (facilities_gdf.geometry.y <= sa_bounds["lat_max"])
]

# Define bounding box geometry for South Africa
south_africa_bbox = box(sa_bounds["lon_min"], sa_bounds["lat_min"], sa_bounds["lon_max"], sa_bounds["lat_max"])

# Load the GeoTIFF file and crop to bounding box
with rasterio.open("/content/drive/MyDrive/MIT805_Exam/population_density.tif") as src:
    # Crop the raster to the bounding box for South Africa
    tiff_data, tiff_transform = mask(src, [south_africa_bbox], crop=True, nodata=np.nan)
    tiff_data = tiff_data[0]  # Extract the first band

# Set color normalization based on the standalone image's settings
vmin_value = np.nanmin(tiff_data)
vmax_value = np.nanpercentile(tiff_data, 95)
norm = mcolors.Normalize(vmin=vmin_value, vmax=vmax_value)

# Apply the normalization and inferno colormap
tiff_data_normalized = norm(tiff_data)

# Set up the plot with a larger figure size
fig, ax = plt.subplots(figsize=(15, 15))

# Plot the GeoTIFF data with the inferno color map for density visualization
cmap = plt.cm.inferno  # High-contrast color map
img = ax.imshow(tiff_data_normalized, cmap=cmap, extent=[tiff_transform[2], tiff_transform[2] + tiff_transform[0] * tiff_data.shape[1],
                                                         tiff_transform[5] + tiff_transform[4] * tiff_data.shape[0], tiff_transform[5]])
plt.colorbar(img, ax=ax, label='Population Density (2020)')

# Overlay health facilities
facilities_gdf.plot(ax=ax, color='blue', markersize=5, label='Health Facilities')

# Additional map details
ax.set_title("Population Density and Health Facilities in South Africa (2020)")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.legend()

# Save the image with a reasonable DPI
plt.savefig("/content/drive/MyDrive/MIT805_Exam/Test.png", dpi=900, bbox_inches='tight')
plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import matplotlib.colors as mcolors
from rasterio.mask import mask
from shapely.geometry import Point, box
import pandas as pd
from PIL import Image

# Load shapefile and facilities data
merged_gdf = gpd.read_file("/content/drive/MyDrive/MIT805_Exam/Shape_Data/zaf_admbnda_adm3_sadb_ocha_20201109.shp")
facilities_df = pd.read_excel("/content/drive/MyDrive/MIT805_Exam/health_facilities.xlsx")
facilities_df = facilities_df[facilities_df['Country'].str.strip().str.lower() == 'south africa']

# Convert facilities_df to a GeoDataFrame with geometry
facilities_df['geometry'] = facilities_df.apply(lambda row: Point(row['Long'], row['Lat']), axis=1)
facilities_gdf = gpd.GeoDataFrame(facilities_df, geometry='geometry')

# Filter facilities_gdf to keep only points within South Africa's bounding box
sa_bounds = {
    "lat_min": -35,
    "lat_max": -22,
    "lon_min": 16,
    "lon_max": 33
}
facilities_gdf = facilities_gdf[
    (facilities_gdf.geometry.x >= sa_bounds["lon_min"]) &
    (facilities_gdf.geometry.x <= sa_bounds["lon_max"]) &
    (facilities_gdf.geometry.y >= sa_bounds["lat_min"]) &
    (facilities_gdf.geometry.y <= sa_bounds["lat_max"])
]

# Define bounding box geometry for South Africa
south_africa_bbox = box(sa_bounds["lon_min"], sa_bounds["lat_min"], sa_bounds["lon_max"], sa_bounds["lat_max"])

# Load and crop the GeoTIFF file to South Africa
with rasterio.open("/content/drive/MyDrive/MIT805_Exam/population_density.tif") as src:
    tiff_data, tiff_transform = mask(src, [south_africa_bbox], crop=True, nodata=np.nan)
    tiff_data = tiff_data[0]  # Extract the first band

# Normalize data and apply the inferno colormap as done in the standalone example
vmin, vmax = 0, np.nanpercentile(tiff_data, 95)
norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
tiff_data_normalized = norm(tiff_data)

# Apply the colormap and convert to RGBA
cmap = plt.cm.inferno  # Using the same colormap as the standalone
rgba_img = cmap(tiff_data_normalized)

# Set pixels with very low values (no data areas) to transparent
alpha_threshold = 0.01  # Set a small threshold for transparency
rgba_img[..., 3] = np.where(tiff_data_normalized > alpha_threshold, 1, 0)  # Alpha=0 for "black" areas

# Convert RGBA data to an image object for plotting
img_overlay = Image.fromarray((rgba_img * 255).astype(np.uint8))

# Plot the combined image with health facilities
fig, ax = plt.subplots(figsize=(15, 15))

# Display the RGBA overlay image with transparency applied
ax.imshow(img_overlay, extent=[tiff_transform[2],
                               tiff_transform[2] + tiff_transform[0] * tiff_data.shape[1],
                               tiff_transform[5] + tiff_transform[4] * tiff_data.shape[0], tiff_transform[5]])

# Overlay health facilities
facilities_gdf.plot(ax=ax, color='blue', markersize=5, label='Health Facilities')

# Additional map details
ax.set_title("Population Density and Health Facilities in South Africa (2020)")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.legend()

# Save the image with a reasonable DPI
plt.savefig("/content/drive/MyDrive/MIT805_Exam/Test_combined_transparent.png", dpi=900, bbox_inches='tight')
plt.show()
